In [46]:
DATASET_PATH="/home/siddhesh/Desktop/Siddhesh/Job Applications/smart-sense/smart-sense-ML-assessment/data/ML-5/Monthly_data_cmo.csv"

## Data Pre-processing and EDA

In [47]:
import pandas as pd
import numpy as np

In [48]:
data=pd.read_csv(DATASET_PATH)
data.shape, data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62429 entries, 0 to 62428
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   APMC             62429 non-null  object
 1   Commodity        62429 non-null  object
 2   Year             62429 non-null  int64 
 3   Month            62429 non-null  object
 4   arrivals_in_qtl  62429 non-null  int64 
 5   min_price        62429 non-null  int64 
 6   max_price        62429 non-null  int64 
 7   modal_price      62429 non-null  int64 
 8   date             62429 non-null  object
 9   district_name    62429 non-null  object
 10  state_name       62429 non-null  object
dtypes: int64(5), object(6)
memory usage: 5.2+ MB


((62429, 11), None)

From the above analysis, we can say that we don't have any null values.

We will predict the model prices, hence remove min_price, max_price column

In the dataset, we can see that the price is determined by the APMC and not by district or state. Also, its important to note that district or state is an larger entity than APMC

In [49]:
data.drop(['min_price','max_price','district_name','state_name','date'],axis=1,inplace=True)
data=data.dropna()
data.isnull().sum()  

APMC               0
Commodity          0
Year               0
Month              0
arrivals_in_qtl    0
modal_price        0
dtype: int64

In [50]:
data.groupby('APMC').count(), data.groupby('Commodity').count()

(               Commodity  Year  Month  arrivals_in_qtl  modal_price
 APMC                                                               
 Aamgaon               27    27     27               27           27
 Aarni                 84    84     84               84           84
 Achalpur             195   195    195              195          195
 Aheri                209   209    209              209          209
 Ahmednagar           649   649    649              649          649
 ...                  ...   ...    ...              ...          ...
 Washim-Ansing         47    47     47               47           47
 Yawal                112   112    112              112          112
 Yeola                192   192    192              192          192
 Yeotmal              175   175    175              175          175
 Zarijamini            52    52     52               52           52
 
 [349 rows x 5 columns],
                  APMC  Year  Month  arrivals_in_qtl  modal_price
 Commodity

### One Hot Encoding

In [51]:
from sklearn.preprocessing import OneHotEncoder  

data = pd.get_dummies(data, columns=['APMC',"Commodity","Month"], prefix = ['APMC',"commodity","month"])  

In [52]:
data.head()

,Year,arrivals_in_qtl,modal_price,APMC_Aamgaon,APMC_Aarni,APMC_Achalpur,APMC_Aheri,APMC_Ahmednagar,APMC_Ahmedpur,APMC_Akhadabalapur,...,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September
0,2015,79,1463,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2016,106,1875,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2015,1253,1731,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2016,387,1999,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015,3825,1900,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Determining Features for the dataset

In [53]:
# Using Min/Max Scaler for the dataset
features=data.loc[:, data.columns !='modal_price']
label=data['modal_price']  

#### Here, we can see that there is significant difference in the magnitude of different features, this may cause the undetermined advantage to the features with more magnitude

In [55]:
from sklearn.preprocessing import MinMaxScaler  

scaler=MinMaxScaler()  
features=scaler.fit_transform(features)  

In [56]:
features

array([[5.00000000e-01, 5.37837191e-05, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 7.24011603e-05, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.00000000e-01, 8.63297645e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.00000000e+00, 3.10290687e-05, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.13773252e-04, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 5.03360448e-05, 0.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])